In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as LA
import scipy
import netket as nk
import jax
import jax.numpy as jnp
import netket.nn as nknn
from jax import grad, jit, vmap, vjp 
from jax import random
from scipy.sparse.linalg import eigsh
from scipy.linalg import eigh, eigvalsh
from sympy.combinatorics import Permutation as Perm
from sympy.interactive import init_printing
import json
import networkx as nx 
from jax import random
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
import netket.nn as nknn
import flax.linen as nn
import time 
import json 




In [ ]:
'''
The 3 x 4 square lattices with frustration  J2 = 0.5 J2 = 0.8
'''

## J2= 0.5

In [ ]:

J = [1, 0.5]
# graph = nk.graph.Grid(extent= [2,4], pbc=False)
# edges = graph.edges
# nx.draw(graph.to_networkx(), with_labels=True, font_weight='bold')
edge_colors = [[0, 1, 1], [0,7, 1], [1, 2,1], [1,6, 1], [2,3,1],
               [2,5,1], [3,4,1], [4,5,1], [4,11,1], [5,6,1], [5, 10, 1],
              [6,9,1], [6,7,1],[7,8,1], [8,9,1], [9,10, 1], [10, 11, 1],
               # J2 terms now for the frustration
               [0,2,2], [0,8,2], [0,6,2], [1,3,2], [1,7,2], [1,9,2],
               [1,5,2], [2,10,2], [2,4,2], [2,6,2], [3,11,2], [3,5,2],
               [4,6,2], [4,10,2], [5,7,2], [5,9,2], [6,8,2], [6, 10, 2], 
               [7,9,2], [8, 10, 2], [9, 11, 2], [5, 11, 2]]

#Sigma^z*Sigma^z interactions
sigmaz = [[1, 0], [0, -1]]
mszsz = (np.kron(sigmaz, sigmaz))

#Exchange interactions
exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])

bond_operator = [
    (J[0] * mszsz).tolist(),
    (J[1] * mszsz).tolist(),
    (J[0] * exchange).tolist(),
    (J[1] * exchange).tolist(),
]

bond_color = [1, 2, 1, 2]

# graph = nk.graph.Grid(extent= [2,4], pbc=False, edge_colors = edge_colors)
g = nk.graph.Graph(edges=edge_colors)
nx.draw(g.to_networkx(), with_labels=True, font_weight='bold')
hi = nk.hilbert.Spin(s=float(0.5), total_sz=float(0.0), N=g.n_nodes)
# print(g.edge_colors)
ha = nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)
evals = nk.exact.lanczos_ed(ha, compute_eigenvectors=False)
exact_gs_energy1 = evals[0]
print('The exact ground-state energy from computational basis for J2 = {} is-- ({}) '.format(J[1], exact_gs_energy1/float(4)))


### Using RBM Model 

In [ ]:
# RBM ansatz with alpha=1
ma = nk.models.RBM(alpha=1, dtype='complex')
# Build the sampler
sa = nk.sampler.MetropolisExchange(hilbert=hi,graph=g, d_max =2)

# Optimizer
op = nk.optimizer.Sgd(learning_rate=0.02)

# Stochastic Reconfiguration
sr = nk.optimizer.SR(diag_shift=float(0.01))

# The variational state
vs = nk.vqs.MCState(sa, ma, n_samples=int(1024))

# The ground-state optimization loop
gs = nk.VMC(
    hamiltonian=ha,
    optimizer=op,
    preconditioner=sr,
    variational_state=vs)

start = time.time()
gs.run(out='RBM', n_iter=int(5000))
end = time.time()

print('### RBM calculation')
print('Has',vs.n_parameters,'parameters')
print('The RBM calculation took',end-start,'seconds')

# import the data from log file
data=json.load(open("RBM.log"))

# Extract the relevant information
iters_RBM1 = data["Energy"]["iters"]
energy_RBM1 = data["Energy"]["Mean"]['real']

### Using GCNN Model 

In [ ]:
symmetries = g.automorphisms()
#Feature dimensions of hidden layers, from first to last
feature_dims = (8,8,8,8)

#Number of layers
num_layers = 4

#Define the GCNN
ma = nk.models.GCNN(symmetries = symmetries, layers = num_layers, 
                    features = feature_dims, dtype='complex')

#Metropois-Hastings with two spins flipped that are at most second nearest neighbors
sa = nk.sampler.MetropolisExchange(hilbert = hi, graph=g, d_max=2)

#Stochaistic reconfiguration
op = nk.optimizer.Sgd(learning_rate=0.02)
sr = nk.optimizer.SR(diag_shift=0.01)

#Define a variational state so we can keep the parameters if we like
vstate = nk.variational.MCState(sampler=sa, model=ma, n_samples=1024)

#Define a driver that performs VMC
gs = nk.driver.VMC(ha, op, sr=sr, variational_state=vstate)
#Run the optimization
gs.run(n_iter=5000, out='out')
#Get data from log and
# energy_gcnn1 = []
# data_gcnn1=json.load(open("out.log"))
# for en in data_gcnn1["Energy"]["Mean"]['real']:
#     energy_gcnn1.append(en)

energy_gcnn1 = []
data_gcnn1=json.load(open("out.log"))
for en in data_gcnn1["Energy"]["Mean"]['real']:
    energy_gcnn1.append(en)
print(min(energy_gcnn1)/4)




In [ ]:
import pandas as pd 
df = pd.read_csv('../data/CQA_J05_6square.csv')
CQA_energy1 = df.to_numpy()[:,1]
CQA_iters1 = np.multiply(df.to_numpy()[:,0], 5.0)

In [ ]:
print(min(CQA_energy1))

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(iters_RBM1, np.divide(energy_RBM1,float(4.0)), color='violet', label='Energy (RBM)')
ax1.plot(iters_RBM1, np.divide(energy_gcnn1,float(4.0)), color='tomato', label='Energy (GCNN)')
ax1.plot(CQA_iters1, CQA_energy1, color='paleturquoise', label='Energy (Sn-CQA)')

ax1.set_ylabel('Energy')
ax1.set_xlabel('Iteration')
ax1.set_ylim([exact_gs_energy1/4 - 0.05, exact_gs_energy1/4 +0.25])
# plt.axis([0,iters_RBM[-1],exact_gs_energy-0.03,exact_gs_energy+0.2])
plt.axhline(y=np.divide(exact_gs_energy1, float(4.0)), xmin=0,
                xmax=iters_RBM1[-1], linewidth=2, color='k', label='Exact=--5.356189')
ax1.legend()
plt.title('3 x 4 Lattice at J2/J1 = 0.5')
plt.show()
fig.savefig('../data/6square_J05.png')

In [ ]:
netket1 = dict()
netket1['GCNN'] = np.divide(energy_gcnn1,float(4.0))
netket1['RBM'] = np.divide(energy_RBM1,float(4.0))
netket1['Exact'] = [exact_gs_energy1/4 for i in range(5000)]

netket_df1 = pd.DataFrame.from_dict(netket1)
netket_df1.to_csv('../data/Netket_J05_6square.csv')

## J2 = 0.8

In [ ]:

J = [1, 0.8]
# graph = nk.graph.Grid(extent= [2,4], pbc=False)
# edges = graph.edges
# nx.draw(graph.to_networkx(), with_labels=True, font_weight='bold')
edge_colors = [[0, 1, 1], [0,7, 1], [1, 2,1], [1,6, 1], [2,3,1],
               [2,5,1], [3,4,1], [4,5,1], [4,11,1], [5,6,1], [5, 10, 1],
              [6,9,1], [6,7,1],[7,8,1], [8,9,1], [9,10, 1], [10, 11, 1],
               # J2 terms now for the frustration
               [0,2,2], [0,8,2], [0,6,2], [1,3,2], [1,7,2], [1,9,2],
               [1,5,2], [2,10,2], [2,4,2], [2,6,2], [3,11,2], [3,5,2],
               [4,6,2], [4,10,2], [5,7,2], [5,9,2], [6,8,2], [6, 10, 2], 
               [7,9,2], [8, 10, 2], [9, 11, 2], [5, 11, 2]]

#Sigma^z*Sigma^z interactions
sigmaz = [[1, 0], [0, -1]]
mszsz = (np.kron(sigmaz, sigmaz))

#Exchange interactions
exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])

bond_operator = [
    (J[0] * mszsz).tolist(),
    (J[1] * mszsz).tolist(),
    (J[0] * exchange).tolist(),
    (J[1] * exchange).tolist(),
]

bond_color = [1, 2, 1, 2]

# graph = nk.graph.Grid(extent= [2,4], pbc=False, edge_colors = edge_colors)
g = nk.graph.Graph(edges=edge_colors)
nx.draw(g.to_networkx(), with_labels=True, font_weight='bold')
hi = nk.hilbert.Spin(s=float(0.5), total_sz=float(0.0), N=g.n_nodes)
# print(g.edge_colors)
ha = nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)
evals = nk.exact.lanczos_ed(ha, compute_eigenvectors=False)
exact_gs_energy2 = evals[0]
print('The exact ground-state energy from computational basis for J2 = {} is-- ({}) '.format(J[1], exact_gs_energy2/float(4)))



### Using RBM

In [ ]:
# RBM ansatz with alpha=1
ma = nk.models.RBM(alpha=1, dtype='complex')
# Build the sampler
sa = nk.sampler.MetropolisExchange(hilbert=hi,graph=g, d_max=2)

# Optimizer
op = nk.optimizer.Sgd(learning_rate=0.02)

# Stochastic Reconfiguration
sr = nk.optimizer.SR(diag_shift=float(0.01))

# The variational state
vs = nk.vqs.MCState(sa, ma, n_samples=int(1024))

# The ground-state optimization loop
gs = nk.VMC(
    hamiltonian=ha,
    optimizer=op,
    preconditioner=sr,
    variational_state=vs)

start = time.time()
gs.run(out='RBM', n_iter=int(5000))
end = time.time()

print('### RBM calculation')
print('Has',vs.n_parameters,'parameters')
print('The RBM calculation took',end-start,'seconds')

# import the data from log file
data=json.load(open("RBM.log"))

# Extract the relevant information
iters_RBM2 = data["Energy"]["iters"]
energy_RBM2 = data["Energy"]["Mean"]["real"]

### Using GCNN 

In [ ]:
symmetries = g.automorphisms()
#Feature dimensions of hidden layers, from first to last
feature_dims = (8,8,8,8)

#Number of layers
num_layers = 4

#Define the GCNN
ma = nk.models.GCNN(symmetries = symmetries, layers = num_layers, features = feature_dims, dtype='complex')

#Metropois-Hastings with two spins flipped that are at most second nearest neighbors
sa = nk.sampler.MetropolisExchange(hilbert = hi, graph=g, d_max=2)

#Stochaistic reconfiguration
op = nk.optimizer.Sgd(learning_rate=0.02)
sr = nk.optimizer.SR(diag_shift=0.01)

#Define a variational state so we can keep the parameters if we like
vstate = nk.variational.MCState(sampler=sa, model=ma, n_samples=1024)

#Define a driver that performs VMC
gs = nk.driver.VMC(ha, op, sr=sr, variational_state=vstate)
#Run the optimization
gs.run(n_iter=5000, out='out')
#Get data from log and


energy_gcnn2 = []
data_gcnn2=json.load(open("out.log"))
for en in data_gcnn2["Energy"]["Mean"]["real"]:
    energy_gcnn2.append(en)




In [ ]:
import pandas as pd 
df = pd.read_csv('../data/CQA_J08_6square2.csv')
CQA_energy2 = df.to_numpy()[:,1]
CQA_iters2 = np.multiply(df.to_numpy()[:,0], 5.0)

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(iters_RBM2, np.divide(energy_RBM2,float(4.0)), color='violet', label='Energy (RBM)')
ax1.plot(iters_RBM2, np.divide(energy_gcnn2,float(4.0)), color='paleturquoise', label='Energy (GCNN)')
ax1.plot(CQA_iters2, CQA_energy2, color='tomato', label='Energy (Sn-CQA)')

ax1.set_ylabel('Energy')
ax1.set_xlabel('Iteration')
ax1.set_ylim([exact_gs_energy2/4 -0.3,exact_gs_energy2/4 +0.3])
# plt.axis([0,iters_RBM[-1],exact_gs_energy-0.03,exact_gs_energy+0.2])
plt.axhline(y=np.divide(exact_gs_energy2, float(4.0)), xmin=0,
                xmax=iters_RBM2[-1], linewidth=2, color='k', label='Exact=-4.8945862')
ax1.legend()
plt.title('12-spin Kagome at J2/J1 = 0.8')
plt.show()
fig.savefig('../data/6square_J08.png')

In [ ]:
netket2 = dict()
netket2['GCNN'] = np.divide(energy_gcnn2,float(4.0))
netket2['RBM'] = np.divide(energy_RBM2,float(4.0))
netket2['Exact'] = [exact_gs_energy2/4 for i in range(5000)]

netket_df2 = pd.DataFrame.from_dict(netket2)
netket_df2.to_csv('../data/Netket_J08_6square.csv')